# 4. Callables in Python

In Python, a **callable** is anything that can be “called” using `(...)`. This most commonly refers to functions (e.g., `def foo(): ...`), but **classes** (via `__call__`), **lambdas**, **functools.partial** objects, and so on can also be callables.

## 4.1 Basic `Callable` Type Hints
To specify that a variable, parameter, or attribute is a function or callable, Python provides the `Callable` type hint. The typical usage is:


In [6]:
from typing import Callable

# A callable that takes two integers and returns a string
MyFuncType = Callable[[int, int], str]

print(MyFuncType)

typing.Callable[[int, int], str]


In [18]:
# Usage
from dataclasses import dataclass
from typing import Callable

@dataclass
class Calculator:
    operation: Callable[[int, int], str]

    def calculate(self, a: int, b: int) -> str:
        return self.operation(a, b)

def add_and_stringify(x: int, y: int) -> str:
    return str(x + y)

calc = Calculator(operation=add_and_stringify)
print(calc.calculate(5, 7))  # Outputs: '12'

12


## 4.2 Generic Callables
Generics let you parameterize a callable’s input or output types using `TypeVar`. For example:

In [24]:
from typing import Callable, TypeVar

T = TypeVar("T")
U = TypeVar("U")

# A generic callable that transforms type T into type U
Transformer = Callable[[T], U]

In [26]:
def apply_transformer(value: T, transformer: Callable[[T], U]) -> U:
    return transformer(value)

# Example 1: Transform an integer into a descriptive string.
def int_to_str(n: int) -> str:
    return f"The number is {n}!"

result1 = apply_transformer(42, int_to_str)
print(result1)  # Outputs: 'The number is 42!'

The number is 42!


#### Usage with Generics and DataClass

In [23]:

from dataclasses import dataclass
from typing import Callable, Generic, TypeVar

# TContext is a type variable used to parameterize our class.
TContext = TypeVar("TContext")

@dataclass
class PotionMixer(Generic[TContext]):
    # The mix function now only expects a context of type TContext and returns a string.
    mix: Callable[[TContext], str]

    def create_potion(self, context: TContext) -> str:
        return self.mix(context)

# Example mixing function that uses a context (here, a dictionary) to create a potion description.
def magical_mix(context: dict) -> str:
    secret = context.get("secret", "moonlight")
    return f"Potion of Wonder with a hint of {secret}!"

# Create an instance of PotionMixer with our magical_mix function.
mixer = PotionMixer(mix=magical_mix)

# Create a potion using a context that defines the secret ingredient.
potion = mixer.create_potion({"secret": "dragon scale"})
print(potion)  # Outputs: 'Potion of Wonder with a hint of dragon scale!'


Potion of Wonder with a hint of dragon scale!



## 4.3 Async vs. Sync Return Types with `MaybeAwaitable`
**Sometimes,** we allow a callable to return **either** a normal (synchronous) value **or** an async `Awaitable` (such as a coroutine). A typical pattern is:

In [40]:
import nest_asyncio
nest_asyncio.apply()

In [43]:
from collections.abc import Awaitable
from typing import Callable, TypeVar, Union

# Define a type variable for the input.
T = TypeVar("T")

# A generic callable that takes a value of type T and returns something you may need to await to yield a string.
MaybeAsyncFunc = Callable[[T], Union[Awaitable[str], str]]

In [44]:
import asyncio
from collections.abc import Awaitable
from typing import Callable, Union

# A callable that takes a candidate's name (str) and returns a MaybeAwaitableStr.
MaybeAsyncLeadAgent = Callable[[str], Union[Awaitable[str], str]]

def lead_generation_agent(candidate: str) -> Union[Awaitable[str], str]:
    """
    - If the candidate's name starts with a vowel, return a qualifying message immediately.
    - Otherwise, simulate further review by returning an awaitable (a coroutine).
    """
    if candidate[0].lower() in "aeiou":
        return f"Lead for {candidate}: Qualified immediately!"
    else:
        async def review_lead() -> str:
            return f"Lead for {candidate}: Requires further review."
        return review_lead()

@dataclass
class LeadGenerationAgent:
  agent_launch_pad: MaybeAsyncLeadAgent

  def __call__(self, candidate: str) -> Union[Awaitable[str], str]:
    return self.agent_launch_pad(candidate)


In [46]:
agent = LeadGenerationAgent(lead_generation_agent)

In [47]:
# Case 1: Candidate whose name starts with a vowel; returns a string immediately.
result1 = agent("Alice")
print("type", type(result1))
if asyncio.iscoroutine(result1):
    result1 = asyncio.run(result1)
print(result1)  # Expected output: "Lead for Alice: Qualified immediately!"

type <class 'str'>
Lead for Alice: Qualified immediately!


In [48]:
# Case 2: Candidate whose name starts with a consonant; returns an awaitable.
result2 = agent("Bob")
print("type", type(result2))
if asyncio.iscoroutine(result2):
    result2 = asyncio.run(result2)
print(result2)  # Expected output: "Lead for Bob: Requires further review."

type <class 'coroutine'>
Lead for Bob: Requires further review.


---

## 4.4 Combining Callables, Generics, and DataClasses
You can embed a callable inside a DataClass, then pass it around in your code. This is powerful in AI agent architectures, where a callable might represent some “dynamic instructions” or “hook” that can be either sync or async.


In [109]:
import asyncio
import inspect
from dataclasses import dataclass
from typing import Generic, TypeVar, Callable, Union, Optional
from collections.abc import Awaitable

TContext = TypeVar("TContext")

@dataclass
class ContextWrapper(Generic[TContext]):
    context: TContext

# Define a type that can either be a static string or a callable that returns instructions.
InstructionProvider = Union[
    str,
    Callable[[ContextWrapper[TContext]], Union[Awaitable[str], str]]
]

@dataclass
class DynamicInstructions(Generic[TContext]):
    instructions: InstructionProvider

    def get_instructions(self, wrapper: Optional[ContextWrapper[TContext]] = None) -> Union[Awaitable[str], str]:
        if callable(self.instructions):
            if wrapper is None:
                raise ValueError("A context must be provided for dynamic instructions")
            result = self.instructions(wrapper)
            # If result is awaitable, ensure it's a coroutine
            if inspect.isawaitable(result):
                # If not a coroutine, wrap it in one.
                if not inspect.iscoroutine(result):
                    async def wrap():
                        return await result
                    result = wrap()
                return asyncio.run(result)
            return result
        return self.instructions

---


### Usage Example

#### a. Pass a string as instructions

In [110]:
# For static instructions, you don't need to pass any context.
static_inst = DynamicInstructions[str](instructions="Write a 100 words tweet on Agentic AI in 2025.")
print(static_inst.get_instructions())

Write a 100 words tweet on Agentic AI in 2025.


#### b. Example with a sync callable

In [111]:
# For dynamic instructions, you'll need to provide a context.
def sync_instruction_provider(ctx: ContextWrapper[str]) -> str:
    return f"Address me as: {ctx.context}"

sync_inst = DynamicInstructions[str](instructions=sync_instruction_provider)
print(sync_inst.get_instructions(ContextWrapper(context="Junaid")))

Address me as: Junaid


#### c. Example with an async callable

In [112]:
async def async_instruction_provider(ctx: ContextWrapper[str]) -> str:
    return f"Address me as: {ctx.context}"

async_inst = DynamicInstructions[str](instructions=async_instruction_provider)

import asyncio
async_inst.get_instructions(ContextWrapper(context="Junaid"))

'Address me as: Junaid'

## 4.5 Real-World Analogy for AI Agents
In more advanced AI agent libraries (like in agents sdk), you often see fields like:

```python
instructions: (
    str
    | Callable[[RunContextWrapper[TContext], Agent[TContext]], MaybeAwaitable[str]]
    | None
) = None
```

This means:
1. **`str`**: a simple, static prompt or instructions text.
2. **`Callable[[RunContextWrapper[TContext], Agent[TContext]], MaybeAwaitable[str]]`**:  
   a function taking two parameters (`RunContextWrapper` and the `Agent` itself) that can return either a `str` (sync) or an async “awaitable” string.
3. **`None`**: no instructions at all.

When the agent runs:
- If `instructions` is a string, it’s used directly.
- If it’s a callable, we call or `await` it to obtain the instructions dynamically.
- If `None`, we might skip or throw an error.

This design approach offers **maximal flexibility**: you can store a static string for simpler use cases, or pass a function that can look at the user’s context, model settings, or any other runtime data to compute a specialized prompt.

---

## Summary

1. **`Callable` Type Hint**: The cornerstone for describing function signatures in typed Python code.
2. **Generics with `Callable`**: Allows you to express “a function that transforms `T` into `U`,” capturing the function’s input and output types for strong type-checking.
3. **`MaybeAwaitable`**: A powerful pattern that supports both synchronous and asynchronous return types, enabling flexible usage in complex AI or concurrency scenarios.
4. **DataClasses + Callables**: You can store callables inside data classes to build more flexible, configurable systems—particularly in AI agent frameworks where instructions, hooks, or tool functions can be dynamic.

By combining these patterns—**Generics**, **DataClasses**, **Protocols**, **Callables**, and **MaybeAwaitable**—we can create highly **type-safe**, **extensible**, and **readable** architectures that scale from simple prototypes to production-grade AI agent systems.